## Generate data list info

We follow the format of `nuscenes` dataset: 

nuscenes_database/xxxxx.bin: point cloud data included in each 3D bounding box of the training dataset

nuscenes_infos_train.pkl: training dataset, a dict contains two keys: metainfo and data_list. metainfo contains the basic information for the dataset itself, such as categories, dataset and info_version, while data_list is a list of dict, each dict (hereinafter referred to as info) contains all the detailed information of single sample as follows:

`info[‘sample_idx’]`: The index of this sample in the whole dataset.

info[‘token’]: Sample data token.

info[‘timestamp’]: Timestamp of the sample data.

info[‘ego2global’]: The transformation matrix from the ego vehicle to global coordinates. (4x4 list)

info[‘lidar_points’]: A dict containing all the information related to the lidar points.

- info[‘lidar_points’][‘lidar_path’]: The filename of the lidar point cloud data.

- info[‘lidar_points’][‘num_pts_feats’]: The feature dimension of point.

- info[‘lidar_points’][‘lidar2ego’]: The transformation matrix from this lidar sensor to ego vehicle. (4x4 list)

info[‘lidar_sweeps’]: A list contains sweeps information (The intermediate lidar frames without annotations)

- info[‘lidar_sweeps’][i][‘lidar_points’][‘data_path’]: The lidar data path of i-th sweep.
    
- info[‘lidar_sweeps’][i][‘lidar_points’][‘lidar2ego’]: The transformation matrix from this lidar sensor to ego vehicle. (4x4 list)
    
- info[‘lidar_sweeps’][i][‘lidar_points’][‘ego2global’]: The transformation matrix from the ego vehicle to global coordinates. (4x4 list)
    
- info[‘lidar_sweeps’][i][‘lidar2sensor’]: The transformation matrix from the main lidar sensor to the current sensor (for collecting the sweep data). (4x4 list)
    
- info[‘lidar_sweeps’][i][‘timestamp’]: Timestamp of the sweep data.
    
- info[‘lidar_sweeps’][i][‘sample_data_token’]: The sweep sample data token.

info[‘images’]: A dict contains six keys corresponding to each camera: 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT'. Each dict contains all data information related to corresponding camera.

- info[‘images’][‘CAM_XXX’][‘img_path’]: The filename of the image.
    
- info[‘images’][‘CAM_XXX’][‘cam2img’]: The transformation matrix recording the intrinsic parameters when projecting 3D points to each image plane. (3x3 list)
    
- info[‘images’][‘CAM_XXX’][‘sample_data_token’]: Sample data token of image.
    
- info[‘images’][‘CAM_XXX’][‘timestamp’]: Timestamp of the image.
    
- info[‘images’][‘CAM_XXX’][‘cam2ego’]: The transformation matrix from this camera sensor to ego vehicle. (4x4 list)
    
- info[‘images’][‘CAM_XXX’][‘lidar2cam’]: The transformation matrix from lidar sensor to this camera. (4x4 list)

info[‘instances’]: It is a list of dict. Each dict contains all annotation information of single instance. For the i-th instance:

- info[‘instances’][i][‘bbox_3d’]: List of 7 numbers representing the 3D bounding box of the instance, in (x, y, z, l, w, h, yaw) order.

- info[‘instances’][i][‘bbox_label_3d’]: A int indicate the label of instance and the -1 indicate ignore.

- info[‘instances’][i][‘velocity’]: Velocities of 3D bounding boxes (no vertical measurements due to inaccuracy), a list has shape (2.).

- info[‘instances’][i][‘num_lidar_pts’]: Number of lidar points included in each 3D bounding box.

- info[‘instances’][i][‘num_radar_pts’]: Number of radar points included in each 3D bounding box.

- info[‘instances’][i][‘bbox_3d_isvalid’]: Whether each bounding box is valid. In general, we only take the 3D boxes that include at least one lidar or radar point as valid boxes.

info[‘cam_instances’]: It is a dict containing keys 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT'. For vision-based 3D object detection task, we split 3D annotations of the whole scenes according to the camera they belong to. For the i-th instance:

- info[‘cam_instances’][‘CAM_XXX’][i][‘bbox_label’]: Label of instance.

- info[‘cam_instances’][‘CAM_XXX’][i][‘bbox_label_3d’]: Label of instance.

- info[‘cam_instances’][‘CAM_XXX’][i][‘bbox’]: 2D bounding box annotation (exterior rectangle of the projected 3D box), a list arrange as [x1, y1, x2, y2].

- info[‘cam_instances’][‘CAM_XXX’][i][‘center_2d’]: Projected center location on the image, a list has shape (2,), .

- info[‘cam_instances’][‘CAM_XXX’][i][‘depth’]: The depth of projected center.

- info[‘cam_instances’][‘CAM_XXX’][i][‘velocity’]: Velocities of 3D bounding boxes (no vertical measurements due to inaccuracy), a list has shape (2,).

- info[‘cam_instances’][‘CAM_XXX’][i][‘attr_label’]: The attr label of instance. We maintain a default attribute collection and mapping for attribute classification.

- info[‘cam_instances’][‘CAM_XXX’][i][‘bbox_3d’]: List of 7 numbers representing the 3D bounding box of the instance, in (x, y, z, l, h, w, yaw) order.

info[‘pts_semantic_mask_path’]：The filename of the lidar point cloud semantic segmentation annotation.


In [59]:
def get_id_cross_reference(root_dir, cross_file):
    """
    Get the cross reference of patient_id between LIDC data and synthetic data
    """
    cross_dict = dict()

    cross_file = f'{root_dir}/{cross_file}'
    with open(cross_file, 'r') as file: 
        for line in file: 
            lidc_ref, data_ref = line.strip().split(",")
            cross_dict[data_ref.strip()] = lidc_ref
    
    return cross_dict



def get_images_path(root_dir, images_dir, patient_id):
    """
    Get image path for each camera
    """

    images_dir = f'{root_dir}/{images_dir}/Patient{patient_id:04}'

    info = dict()

    for cam in range(10):
        cam_name = (f'CAM_{cam:02}:').upper()
        info[cam_name] = f'{images_dir}/Image_{cam:02}.png'
    
    return info


def get_3d_annotation(root_dir, anno3d_dir, patient_id):
    """
    Get 3d annotation from raw .txt file
    """

    anno_3d_file = f'{root_dir}/{anno3d_dir}/Patient_{patient_id:04}_bbox3d.txt'

    instances = []
    with open(anno_3d_file, "r") as file:
        for line in file:
            instance_data = {}
            # Convert the comma-separated values into floats
            row = [float(value) for value in line.strip().split(",")]
            row.append(0)  # add yaw value

            instance_data['bbox_3d'] = row
            instance_data['bbox_lavel_3d'] = 1
            instance_data['bbox_3d_isvalid'] = True
            instance_data['num_lidar_pts'] = 0
            instance_data['num_rader_pts'] = 0
            instance_data['velocity'] = [0.0, 0.0]

            instances.append(instance_data)
    return instances


def get_2d_annotation(root_dir, anno2d_dir, patient_id):
    """
    Get 2d annotation from raw .txt file
    """
    anno_2d_dir = f'{root_dir}/{anno2d_dir}/Patient{patient_id:04}'

    cam_instances = dict()

    for cam in range(10):
        bbox2d_file_path = f'{anno_2d_dir}/Cam_{cam:02}_bbox2d.txt'
        cam_name = (f'CAM_{cam:02}:').upper()

        with open(bbox2d_file_path, "r") as file:
            cam_instance = []
            for line in file:
                instance_data = {}
                row = [value for value in line.strip().split(",")]
                instance_data['bbox'] = row[:4]
                instance_data['bbox_label'] = 1
                instance_data['bbox_label_3d'] = 1

            cam_instance.append(instance_data)
        cam_instances[cam_name] = cam_instance
    return cam_instances


In [ ]:
# MAIN

import os
from os import path as osp
import pickle

# Arguments
root_path = 'data/LIDC_Projection_Dataset'
# Root directory to start walking
# since we're in sandbox folder. 
root_dir = f'../{root_path}'

info_prefix = 'lidc'
version = 'v1.0'
dataset_name = ' lidc'
out_dir = root_dir
images_dir = 'Images'
anno_3d_dir = 'Labels3d'
anno_2d_dir = 'Labels2d'


db_info_save_path = osp.join(out_dir, f'{info_prefix}_dbinfos.pkl')
info_train_path = osp.join(out_dir, f'{info_prefix}_dbinfos_train.pkl')
info_val_path = osp.join(out_dir, f'{info_prefix}_dbinfos_val.pkl')
error_log_path = osp.join(out_dir, f'{info_prefix}_error_logs.txt')

# Get the cross reference
cross_file = 'patients_processed.txt'
cross_ref = get_id_cross_reference(root_dir, cross_file)
nb_patient = len(cross_ref)


# Initalize list
all_db_infos = dict()  # Store all data info
logs = dict()  # Store error logs (if any)


# Build metainfo of the dataset
metainfo = {
    'categories': {'normal': 0,'nodule': 1},
    'dataset': 'lidc',
    'version': 'v1.0',
    'info_version': '1.0'
}
all_db_infos['metainfo'] = metainfo


# Build the ground truth database
all_db_datalist = []  # Store all datalist (inside db_infos)

for i in range(nb_patient):

    info_data = dict()

    try: 
        # Build patient_id meta data 
        info_data['sample_id'] = i
        info_data['lidc_id_ref'] = cross_ref[str(i)]

        # Build Images paths
        info_data['images'] = get_images_path(root_dir, images_dir=images_dir, patient_id=i)

        # Build 3D annotation paths
        info_data['instances'] = get_3d_annotation(root_dir, anno_3d_dir, i)

        # Build 2D annotation paths
        info_data['cam_instances'] = get_2d_annotation(root_dir, anno_2d_dir, i)

        # Write to datalist
        all_db_datalist.append(info_data)

    except Exception as e: 
        logs[f'Patient_{i:04}'] = str(e)
        continue

all_db_infos['data_list'] = all_db_datalist

# Write to disk
with open(db_info_save_path, 'wb') as f:
    pickle.dump(all_db_infos, f)

# Write error log file
with open(error_log_path, 'w') as f:
    for key, value in logs.items():
        f.write(f'{key}, {value}\n')

# Process data infos all
 

In [60]:
with open(db_info_save_path, 'rb') as f:
    db_all_infos = pickle.load(f)

len(db_all_infos['data_list'])

790

In [96]:
import mmcv
import numpy as np

data = db_all_infos['data_list']
bbox_test = data[0]['cam_instances']['CAM_00:'][0]['bbox']  # store as a list
bbox_test = np.array([[0, 0, 50, 50], [20, 20, 60, 60]])
img_test = data[0]['images']['CAM_00:']

In [109]:
# mmcv.imshow(img_test)
mmcv.imshow_bboxes(img_test, bbox_test)

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       [[  0, 255,   0],
        [127, 127, 127],
        [127, 127, 127],
        ...,
        [127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]],

       [[  0, 255,   0],
        [126, 126, 126],
        [126, 126, 126],
        ...,
        [127, 127, 127],
        [126, 126, 126],
        [126, 126, 126]],

       ...,

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [129, 129, 129],
        [128, 128, 128],
        [128, 128, 128]],

       [[129, 129, 129],
        [129, 129, 129],
        [129, 129, 129],
        ...,
        [129, 129, 129],
        [129, 129, 129],
        [129, 129, 129]],

       [[129, 129, 129],
        [129, 129, 129],
        [129, 129, 129],
        ...,
        [129, 129, 129],
        [129, 129, 129],
        [129, 129, 129]]

In [ ]:
import os

# Path to the folder
folder_path = f'{root_dir}/Labels3d'

# find all Patient that has bbox_3d
txt_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]
txt_files.sort()
patient_idx = [int(file.split('_')[1]) for file in txt_files]

# find patient that does not have bbox3d
patient_no_bbox3d = list(set(range(911)) - set(patient_idx))
patient_no_bbox3d.sort()

for i in patient_idx:

    info = dict()
    info['sample_idx'] = i

    images_dir = f'{root_dir}/Images/Patient{i:04}'
    anno_2d_dir = f'{root_dir}/Labels2d/Patient{i:04}'
    anno_3d_file = f'{root_dir}/Labels3d/Patient_{i:04}_bbox3d.txt'

    info['images'] = get_images_path(images_dir)
    info['instances'] = get_3d_annotation(anno_3d_file)
    info['cam_instances'] = get_2d_annotation(anno_2d_dir)
    
    data_list.append(info)
